## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config5 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database\WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Percent Humidity,Percent Cloudiness,Wind Speed,Current Description
0,0,Hamilton,39.1834,-84.5333,78.08,41,1,1.01,clear sky
1,1,Yellowknife,62.4560,-114.3525,45.41,74,75,4.00,broken clouds
2,2,Vaini,-21.2000,-175.2000,73.56,83,75,3.44,broken clouds
3,3,Salair,54.2312,85.7972,30.97,99,100,6.78,snow
4,4,Caxito,-8.5785,13.6643,75.85,82,65,4.18,broken clouds
5,5,Olafsvik,64.8945,-23.7142,40.41,87,100,42.66,light rain
6,6,Isla Mujeres,21.2311,-86.7310,87.85,69,19,8.25,few clouds
7,7,Hilo,19.7297,-155.0900,80.26,90,90,5.75,light rain
8,8,Port Keats,-14.2500,129.5500,75.74,84,99,5.17,overcast clouds
9,9,Punta Arenas,-53.1500,-70.9167,48.31,53,40,18.41,scattered clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("what is the minimum temperature you would like for your vacation?"))
max_temp = float(input("what is the maximum temperature you would like for your vacation?"))

what is the minimum temperature you would like for your vacation?70
what is the maximum temperature you would like for your vacation?95


In [24]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Percent Humidity,Percent Cloudiness,Wind Speed,Current Description
0,0,Hamilton,39.1834,-84.5333,78.08,41,1,1.01,clear sky
2,2,Vaini,-21.2000,-175.2000,73.56,83,75,3.44,broken clouds
4,4,Caxito,-8.5785,13.6643,75.85,82,65,4.18,broken clouds
6,6,Isla Mujeres,21.2311,-86.7310,87.85,69,19,8.25,few clouds
7,7,Hilo,19.7297,-155.0900,80.26,90,90,5.75,light rain
8,8,Port Keats,-14.2500,129.5500,75.74,84,99,5.17,overcast clouds
11,11,Lorengau,-2.0226,147.2712,80.89,82,100,9.24,overcast clouds
12,12,Kokopo,-4.3520,152.2633,76.53,84,94,4.14,light rain
15,15,Kapaa,22.0752,-159.3190,82.38,76,75,5.99,light rain
16,16,Dolores,-36.3132,-57.6792,70.14,69,65,7.09,broken clouds


In [26]:
len(preferred_cities_df)

330

In [30]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Percent Humidity       0
Percent Cloudiness     0
Wind Speed             0
Current Description    0
dtype: int64

In [33]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
clean_df = preferred_cities_df

In [35]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hamilton,78.08,clear sky,39.1834,-84.5333,
2,Vaini,73.56,broken clouds,-21.2000,-175.2000,
4,Caxito,75.85,broken clouds,-8.5785,13.6643,
6,Isla Mujeres,87.85,few clouds,21.2311,-86.7310,
7,Hilo,80.26,light rain,19.7297,-155.0900,
8,Port Keats,75.74,overcast clouds,-14.2500,129.5500,
11,Lorengau,80.89,overcast clouds,-2.0226,147.2712,
12,Kokopo,76.53,light rain,-4.3520,152.2633,
15,Kapaa,82.38,light rain,22.0752,-159.3190,
16,Dolores,70.14,broken clouds,-36.3132,-57.6792,


In [65]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = float('NaN')
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [66]:
hotel_df.head(10)

,City,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hamilton,78.08,clear sky,39.1834,-84.5333,Six Acres Bed & Breakfast
2,Vaini,73.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
4,Caxito,75.85,broken clouds,-8.5785,13.6643,NaN
6,Isla Mujeres,87.85,few clouds,21.2311,-86.7310,Hotel La Joya Isla Mujeres
7,Hilo,80.26,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
8,Port Keats,75.74,overcast clouds,-14.2500,129.5500,TDC Village (Hundred Man Camp)
11,Lorengau,80.89,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
12,Kokopo,76.53,light rain,-4.3520,152.2633,Kokopo Beach Bungalow Resort
15,Kapaa,82.38,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
16,Dolores,70.14,broken clouds,-36.3132,-57.6792,Apart Hotel - Arianos


In [69]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hamilton,78.08,clear sky,39.1834,-84.5333,Six Acres Bed & Breakfast
2,Vaini,73.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
6,Isla Mujeres,87.85,few clouds,21.2311,-86.7310,Hotel La Joya Isla Mujeres
7,Hilo,80.26,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
8,Port Keats,75.74,overcast clouds,-14.2500,129.5500,TDC Village (Hundred Man Camp)
...,...,...,...,...,...,...
675,Nago,82.56,scattered clouds,26.5881,127.9761,Hotel Yugaf Inn Okinawa
676,Popondetta,74.82,broken clouds,-8.7537,148.2534,Timos Transit House
677,Ca Mau,76.73,overcast clouds,9.1769,105.1500,Khách Sạn Mường Thanh Luxury Cà Mau
678,Vung Tau,80.92,broken clouds,10.3460,107.0843,Green hotel


In [76]:
# 8a. Create the output File (CSV)
output_data_file = "..\Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [78]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [81]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))